In [1]:
# 1. 调用必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# 2. define hyperparameters
BATCH_SIZE = 512 # 每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu') # cpu训练，可选GPU 
EPOCHS = 10 # 训练10轮

In [3]:
# 3. transform
TRANSFORM = transforms.Compose([
    transforms.ToTensor(), #将图片转换为tensor
    transforms.Normalize((0.1307,), (0.3081,)) #正则化，降低复杂度，更好generalization
])

In [4]:
# 4. downloads datasets
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=TRANSFORM) # 运用pytorch内置MNIST数据集

test_set = datasets.MNIST("data", train=False, download=True, transform=TRANSFORM)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 5. 构建网络模型
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 5) # 1: 灰度图片的通道， 10：输出通道， 5：kernel
        self.conv2 = nn.Conv2d(10, 20, 3) # 10：输入通道 20：输出通道， 3：kernel大小
        self.fc1 = nn.Linear(20*10*10, 500) # 20*10*10：输入通道，500：输出通道
        self.fc2 = nn.Linear(500, 10) # 500：输入通道，10：输出通道
    
    def forward(self, x):
        input_size = x.size(0) # batch_size * 1 * 28 * 28
        x = self.conv1(x) # 输入：batch_size * 1 * 28 * 28, 输入：batch_size * 10* 24 * 24
        x = F.relu(x) # 激活函数RELU，shape不变
        x = F.max_pool2d(x, 2, 2) # 最大池化层，输入：batch_size*10*24*24，输出：batch*10*12*12
        
        x = self.conv2(x) # 输入：batch_size*10*12*12 输出：batch_size*20*10*10
        x = F.relu(x)
        
        x = x.view(input_size, -1) # 拉平，-1：自动计算维度，20*10*10 =2000
        
        x = self.fc1(x) # 输入：batch_size*2000 输出：batch_size*500
        x = F.relu(x)
        
        x = self.fc2(x) # 输入：batch_size*500 输出：batch_size*10
        
        output = F.log_softmax(x, dim=1) # 计算分类后每个数据的概率值
        
        return output

In [6]:
# 6. 定义优化器
model = Net().to(DEVICE)

optimizer = optim.Adam(model.parameters()) # 使用Adam优化器

In [7]:
# 7 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 开始训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 数据部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 预测
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 找到概率值最大的下标
        pred = output.argmax(dim=1) 
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 10 == 0:
            print("Train Epoch : {}, index : {} \t Loss : {:.6f}".format(epoch, batch_index, loss.item()))
            

In [8]:
# 8 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad(): # 不计算梯度和反向传播
        for data, target in test_loader:
            # 部署到device上
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1]
            # 累积正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test Average loss : {:.4f}, Accuracy : {:.3f}\n".format(
              test_loss, 100.0 * correct / len(test_loader.dataset)))
            

In [9]:
# 9 调用方法
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch : 1, index : 0 	 Loss : 2.307470
Train Epoch : 1, index : 10 	 Loss : 0.564772
Train Epoch : 1, index : 20 	 Loss : 0.466786
Train Epoch : 1, index : 30 	 Loss : 0.291659
Train Epoch : 1, index : 40 	 Loss : 0.308937
Train Epoch : 1, index : 50 	 Loss : 0.223918
Train Epoch : 1, index : 60 	 Loss : 0.206420
Train Epoch : 1, index : 70 	 Loss : 0.174509
Train Epoch : 1, index : 80 	 Loss : 0.151677
Train Epoch : 1, index : 90 	 Loss : 0.137973
Train Epoch : 1, index : 100 	 Loss : 0.129320
Train Epoch : 1, index : 110 	 Loss : 0.119090
Test Average loss : 0.0002, Accuracy : 97.090

Train Epoch : 2, index : 0 	 Loss : 0.074841
Train Epoch : 2, index : 10 	 Loss : 0.076890
Train Epoch : 2, index : 20 	 Loss : 0.100701
Train Epoch : 2, index : 30 	 Loss : 0.071623
Train Epoch : 2, index : 40 	 Loss : 0.053419
Train Epoch : 2, index : 50 	 Loss : 0.064680
Train Epoch : 2, index : 60 	 Loss : 0.065044
Train Epoch : 2, index : 70 	 Loss : 0.070620
Train Epoch : 2, index : 80 	 Los